In [ ]:
import os
import random

# Base paths to both datasets
base_paths = [
    "sampledata/AugmentedAlzheimerDataset",
    "sampledata/OriginalDataset"
]

# Target number of images per folder
TARGET_COUNT = 500

for base_path in base_paths:
    print(f"\n{'='*50}")
    print(f"Processing: {base_path}")
    print('='*50)
    
    # Get all subdirectories (MildDemented, ModerateDemented, etc.)
    subdirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

    for subdir in subdirs:
        folder_path = os.path.join(base_path, subdir)
        
        # Get all image files in the folder
        images = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
        
        current_count = len(images)
        print(f"{subdir}: {current_count} images found")
        
        if current_count > TARGET_COUNT:
            # Calculate how many to delete
            num_to_delete = current_count - TARGET_COUNT
            
            # Randomly select images to delete
            images_to_delete = random.sample(images, num_to_delete)
            
            # Delete the selected images
            for img in images_to_delete:
                img_path = os.path.join(folder_path, img)
                os.remove(img_path)
            
            print(f"  -> Deleted {num_to_delete} images. Now has {TARGET_COUNT} images.")
        else:
            print(f"  -> Already has {current_count} images (≤ {TARGET_COUNT}). No deletion needed.")

print("\nDone! All folders now have at most 500 images.")

MildDemented: 8960 images found
  -> Deleted 8460 images. Now has 500 images.
ModerateDemented: 6464 images found
  -> Deleted 8460 images. Now has 500 images.
ModerateDemented: 6464 images found
  -> Deleted 5964 images. Now has 500 images.
  -> Deleted 5964 images. Now has 500 images.
NonDemented: 9600 images found
NonDemented: 9600 images found
  -> Deleted 9100 images. Now has 500 images.
VeryMildDemented: 8960 images found
  -> Deleted 9100 images. Now has 500 images.
VeryMildDemented: 8960 images found
  -> Deleted 8460 images. Now has 500 images.

Done! All folders now have at most 200 images.
  -> Deleted 8460 images. Now has 500 images.

Done! All folders now have at most 200 images.
